## Working with CTKG in Deep Graph Library (DGL)
This notebook provides an example of building a heterograph from CTKG in DGL; and some examples of queries on the DGL heterograph. For more information about using DGL please refer to https://www.dgl.ai/ "
This notebook builds on the notebook from DRKG: https://github.com/gnn4dr/DRKG

In [1]:
import pandas as pd
import numpy as np
import dgl
import sys
sys.path.insert(1, 'utils')
from utils import download_and_extract
#download_and_extract()
ctkg_file = '/Users/gopinath.balu/Workspace/agentic_ai_innovations/CTKG/ctkg.tsv'
df = pd.read_csv(ctkg_file, sep ="\t", header=None)
triplets = df.values.tolist()

FileNotFoundError: Cannot find DGL C++ graphbolt library at /Users/gopinath.balu/Workspace/agentic_ai_innovations/.venv/lib/python3.11/site-packages/dgl/graphbolt/libgraphbolt_pytorch_2.3.1.dylib

In [12]:
!pip freeze

appnope==0.1.4
asttokens==3.0.0
certifi==2025.8.3
charset-normalizer==3.4.2
comm==0.2.3
debugpy==1.8.15
decorator==5.2.1
dgl==2.2.0
executing==2.2.0
filelock==3.18.0
fsspec==2025.7.0
idna==3.10
ipykernel==6.30.1
ipython==9.4.0
ipython_pygments_lexers==1.1.1
jedi==0.19.2
Jinja2==3.1.6
jupyter_client==8.6.3
jupyter_core==5.8.1
MarkupSafe==3.0.2
matplotlib-inline==0.1.7
mpmath==1.3.0
nest-asyncio==1.6.0
networkx==3.5
numpy==2.3.2
packaging==25.0
pandas==2.3.1
parso==0.8.4
pexpect==4.9.0
pillow==11.3.0
platformdirs==4.3.8
prompt_toolkit==3.0.51
psutil==7.0.0
ptyprocess==0.7.0
pure_eval==0.2.3
Pygments==2.19.2
python-dateutil==2.9.0.post0
pytz==2025.2
pyzmq==27.0.1
requests==2.32.4
scipy==1.16.1
six==1.17.0
stack-data==0.6.3
sympy==1.14.0
torch==2.3.1
torchaudio==2.3.1
torchdata==0.11.0
torchvision==0.18.1
tornado==6.5.1
tqdm==4.67.1
traitlets==5.14.3
typing_extensions==4.14.1
tzdata==2025.2
urllib3==2.5.0
wcwidth==0.2.13


In [13]:
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [10]:
# !pip install pandas numpy 

In [8]:
# !pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1

In [9]:
# !pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/repo.html

Assign an ID to each node (entity): create a dictionary of node-types: each dictionary further consists of a dictionary mapping node to an ID.

In [ ]:
entity_dictionary = {}
def insert_entry(entry, ent_type, dic):
    if ent_type not in dic:
        dic[ent_type] = {}
    ent_n_id = len(dic[ent_type])
    if entry not in dic[ent_type]:
         dic[ent_type][entry] = ent_n_id
    return dic

for triple in triplets:
    src = triple[0]
    split_src = src.split('::')
    src_type = split_src[0]
    dest = triple[2]
    split_dest = dest.split('::')
    dest_type = split_dest[0]
    insert_entry(src,src_type,entity_dictionary)
    insert_entry(dest,dest_type,entity_dictionary)

Create a dictionary of relations: the key is the relation and the value is the list of (source node ID, destimation node ID) tuples.

In [ ]:
edge_dictionary={}
for triple in triplets:
    src = triple[0]
    split_src = src.split('::')
    src_type = split_src[0]
    dest = triple[2]
    split_dest = dest.split('::')
    dest_type = split_dest[0]
    
    src_int_id = entity_dictionary[src_type][src]
    dest_int_id = entity_dictionary[dest_type][dest]
    
    pair = (src_int_id,dest_int_id)
    etype = (src_type,triple[1],dest_type)
    if etype in edge_dictionary:
        edge_dictionary[etype] += [pair]
    else:
        edge_dictionary[etype] = [pair]

## Create a DGL heterograph using the dictionary of relations

In [ ]:
graph = dgl.heterograph(edge_dictionary);

## Print the statistics of the created graph

Number of nodes for each node-type

In [ ]:
total_nodes = 0;
for ntype in graph.ntypes:
    print(ntype, '\t', graph.number_of_nodes(ntype));
    total_nodes += graph.number_of_nodes(ntype);
print("Graph contains {} nodes from {} node-types.".format(total_nodes, len(graph.ntypes)))

Number of edges for each relation (edge-type)

In [ ]:
total_edges = 0;
for etype in graph.etypes:
    print(etype, '\t', graph.number_of_edges(etype))
    total_edges += graph.number_of_edges(etype);
print("Graph contains {} edges from {} edge-types.".format(total_edges, len(graph.etypes)))

Just printing the graph ("print(graph)") will also print the graph summary

In [ ]:
print(graph)

In [1]:
from datasets import load_dataset

dataset_pubmed = load_dataset('Pubmed')

/Users/gopinath.balu/Workspace/agentic_ai_innovations/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DataFilesNotFoundError: No (supported) data files found in Pubmed